In [1]:
import requests, json

resp = requests.post(
    "http://localhost:11434/api/pull",
    json={"name": "mistral"},
    stream=True,
)

for line in resp.iter_lines():
    if not line:
        continue
    obj = json.loads(line.decode("utf-8"))
    # Print progress updates without spamming too hard
    if "status" in obj:
        if "completed" in obj and "total" in obj:
            print(obj["status"], obj["completed"], "/", obj["total"])
        else:
            print(obj["status"])

pulling manifest
pulling f5074b1221da 4372811712 / 4372811712
pulling 43070e2d4e53 11356 / 11356
pulling 1ff5b64b61b9 799 / 799
pulling ed11eda7790d 30 / 30
pulling 1064e17101bd 487 / 487
verifying sha256 digest
writing manifest
success


In [2]:
from pathlib import Path
import os

# Ensure we're running from project root
ROOT = Path.cwd()
if ROOT.name == "src":
    ROOT = ROOT.parent

os.chdir(ROOT)
print("Working directory:", Path.cwd())

Working directory: /Users/ryanbrowder/Documents/Projects/kingKillerBot


In [3]:
BOOK_CODE = "NOTW"

In [4]:
import json
from pathlib import Path

import faiss
from sentence_transformers import SentenceTransformer

# --- Paths (root-relative) ---
INDEX_PATH = Path("data/index/NOTW.faiss")
META_PATH  = Path("data/index/NOTW_meta.jsonl")

MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
TOP_K = 60  # retrieve more than we need, then filter

def load_metadata(path: Path):
    rows = []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            rows.append(json.loads(line))
    return rows

def retrieve(
    query: str,
    max_chapter: int,
    top_k: int = 80,
    min_results: int = 12,
    max_top_k: int = 500,
    debug: bool = False
):
    index = faiss.read_index(str(INDEX_PATH))
    meta = load_metadata(META_PATH)

    model = SentenceTransformer(MODEL_NAME)
    q_emb = model.encode(
        [query],
        convert_to_numpy=True,
        normalize_embeddings=True,
    )

    debug_rows = []
    results = []

    k = top_k
    while True:
        scores, indices = index.search(q_emb, k)

        results = []
        debug_rows = []

        for rank, (score, idx) in enumerate(zip(scores[0], indices[0]), start=1):
            row = meta[idx]

            passes = row.get("chapter", 10**9) <= max_chapter
            if debug:
                preview = row.get("text", "")[:250].replace("\n", " ")
                debug_rows.append({
                    "rank": rank,
                    "idx": int(idx),
                    "score": float(score),
                    "chapter": row.get("chapter"),
                    "passes_chapter_filter": passes,
                    "preview": preview
                })

            if passes:
                r = dict(row)
                r["score"] = float(score)
                r["idx"] = int(idx)
                results.append(r)

        # sort safe results
        results.sort(key=lambda x: x["score"], reverse=True)

        # ✅ stop if we have enough safe chunks OR we’ve searched deep enough
        if len(results) >= min_results or k >= max_top_k:
            break

        # otherwise search deeper
        k = min(k * 2, max_top_k)

    if debug:
        dbg = {
            "query": query,
            "max_chapter": max_chapter,
            "min_results": min_results,
            "final_k": k,
            "returned_after_filter": len(results),
            "rows": debug_rows
        }
        return results, dbg

    return results

In [5]:
import re
import hashlib

# -------------------------
# Formatting / helpers
# -------------------------
def format_context(chunks, max_chars=3000, per_chunk_chars=450, max_chunks=6):
    context_parts = []
    used = 0
    count = 0

    for c in chunks:
        if count >= max_chunks:
            break

        text = (c.get("text") or "").strip()
        if len(text) > per_chunk_chars:
            text = text[:per_chunk_chars].rstrip() + "…"

        score = c.get("score", None)
        score_str = f"{score:.3f}" if isinstance(score, (int, float)) else "n/a"
        tag = "expanded" if c.get("expanded") else "chunk"

        block = f"[Chapter {c.get('chapter')}] ({tag}, score={score_str})\n{text}"

        if used + len(block) + 2 > max_chars:
            break

        context_parts.append(block)
        used += len(block) + 2
        count += 1

    return "\n\n".join(context_parts).strip()


def _normalize(txt: str) -> str:
    return re.sub(r"\s+", " ", (txt or "").strip()).lower()


def _fingerprint(txt: str) -> str:
    t = _normalize(txt)
    core = (t[:600] + "||" + t[-600:]) if len(t) > 1200 else t
    return hashlib.md5(core.encode("utf-8")).hexdigest()


def _chunk_id(c):
    return (c.get("chapter"), _fingerprint(c.get("text", "")))


def merge_triplet(prev_c, c, next_c, max_chars=1400):
    parts = []
    for x in (prev_c, c, next_c):
        if not x:
            continue
        t = (x.get("text") or "").strip()
        if t:
            parts.append(t)

    merged = "\n\n".join(parts).strip()
    if len(merged) > max_chars:
        merged = merged[:max_chars].rstrip() + "…"

    out = dict(c)
    out["text"] = merged
    out["score"] = c.get("score")
    out["expanded"] = True
    return out


_STOP = {
    "the","and","for","with","that","this","what","why","does","did","his","her","their",
    "from","into","then","than","have","has","had","not","but","are","was","were","you",
    "your","they","them","who","when","where","how"
}


def _query_terms(q: str):
    terms = re.findall(r"[a-z']{3,}", (q or "").lower())
    return {t for t in terms if t not in _STOP}


def _overlap_score(chunk_text: str, q_terms: set) -> int:
    t = (chunk_text or "").lower()
    return sum(1 for term in q_terms if term in t)


def _expand_queries(question: str):
    q = (question or "").strip()
    return [q, f"{q} explanation", f"{q} reason"]


# -------------------------
# Context packing
# -------------------------
def pack_for_context(chunks, q_terms, min_overlap=1, target=14, min_hits=4):
    def split(threshold):
        hit, miss = [], []
        for c in chunks:
            o = _overlap_score(c.get("text", ""), q_terms)
            (hit if o >= threshold else miss).append(c)
        return hit, miss

    hit, miss = split(min_overlap)

    if len(hit) < min_hits and min_overlap > 0:
        hit, miss = split(0)

    return (hit + miss)[:target]


# -------------------------
# Main entry
# -------------------------
def ask(question: str, current_chapter: int, show_debug: bool = False):
    def dbg(*args):
        if show_debug:
            print(*args)

    expanded_queries = _expand_queries(question)
    q_terms = _query_terms(" ".join(expanded_queries))

    alpha = 0.10
    def _rank_score(c):
        s = c.get("score", 0.0) or 0.0
        o = _overlap_score(c.get("text", ""), q_terms)
        return s + alpha * o

    all_chunks = []
    all_debug_rows = []
    final_k_max = 0
    returned_after_filter_sum = 0

    for q in expanded_queries:
        result = retrieve(
            q,
            max_chapter=current_chapter,
            top_k=TOP_K,
            min_results=25,
            max_top_k=800,
            debug=show_debug
        )

        if show_debug:
            chunks, dbg_info = result
            if dbg_info:
                final_k_max = max(final_k_max, dbg_info.get("final_k", 0) or 0)
                returned_after_filter_sum += dbg_info.get("returned_after_filter", 0) or 0
                all_debug_rows.extend(dbg_info.get("rows", [])[:30])
        else:
            chunks = result

        all_chunks.extend(chunks or [])

    dbg(f"[DEBUG] multiquery final_k_max={final_k_max} | safe_after_filter_sum={returned_after_filter_sum}")

    if not all_chunks:
        return {
            "question": question,
            "current_chapter": current_chapter,
            "context": "",
            "note": "No spoiler-safe context available.",
            "debug": None
        }

    # -------------------------
    # Dedup best-wins
    # -------------------------
    best = {}
    for c in all_chunks:
        cid = _chunk_id(c)
        if cid not in best or _rank_score(c) > _rank_score(best[cid]):
            best[cid] = c

    deduped = sorted(best.values(), key=_rank_score, reverse=True)
    dbg("[DEBUG] top10 deduped:", [(c.get("chapter"), round(c.get("score") or 0, 3)) for c in deduped[:10]])

    # -------------------------
    # Expand anchors
    # -------------------------
    meta = load_metadata(META_PATH)

    def _get_safe(i):
        if 0 <= i < len(meta):
            row = meta[i]
            if row.get("chapter", 10**9) <= current_chapter:
                r = dict(row)
                r["idx"] = i
                r["score"] = None
                return r
        return None

    expanded = []
    for c in deduped[:12]:
        if c.get("idx") is not None:
            expanded.append(merge_triplet(
                _get_safe(c["idx"] - 1),
                c,
                _get_safe(c["idx"] + 1),
                max_chars=1600
            ))
        else:
            expanded.append(c)

    # -------------------------
    # Final dedupe (prefer expanded)
    # -------------------------
    final = {}
    for c in expanded:
        cid = _chunk_id(c)
        if cid not in final or (
            c.get("expanded") and not final[cid].get("expanded")
        ):
            final[cid] = c

    final_chunks = sorted(final.values(), key=_rank_score, reverse=True)

    dbg("[DEBUG] final_chunks top:",
        [(c.get("chapter"), round(c.get("score") or 0, 3), bool(c.get("expanded")))
         for c in final_chunks[:8]])

    final_chunks = pack_for_context(
        final_chunks,
        q_terms,
        target=max(14, 3 * 6)
    )

    context = format_context(final_chunks)

    return {
        "question": question,
        "current_chapter": current_chapter,
        "context": context,
        "note": "",
        "debug": {
            "rows": all_debug_rows,
            "final_k": final_k_max,
            "returned_after_filter": returned_after_filter_sum
        } if show_debug else None
    }

In [6]:
def build_prompt(question: str, current_chapter: int, context: str) -> dict:
    system = f"""
You are a spoiler-safe companion for The Kingkiller Chronicle.
The user is currently at Chapter {current_chapter} of {BOOK_CODE}.

Rules:
- Do NOT use or imply knowledge from after Chapter {current_chapter}.
- Use ONLY the provided context to answer. If context is insufficient, say so.
- Be clear and direct. Base your answer strictly on the context. 
- If something is implied rather than stated, say so explicitly.
- If the question depends on later chapters, say: "I can’t answer that yet without spoilers."
""".strip()

    user = f"""
Question:
{question}

Context (spoiler-safe excerpts up to Chapter {current_chapter}):
{context}
""".strip()

    return {"system": system, "user": user}

In [7]:
import requests

def ollama_chat(system: str, user: str, model: str = "mistral") -> str:
    r = requests.post(
        "http://localhost:11434/api/chat",
        json={
            "model": model,
            "messages": [
                {"role": "system", "content": system},
                {"role": "user", "content": user},
            ],
            "stream": False,
        },
        timeout=120,
    )
    r.raise_for_status()
    return r.json()["message"]["content"].strip()

In [8]:
import re

def answer(question: str, current_chapter: int, model="mistral"):
    resp = ask(question, current_chapter=current_chapter)

    if not resp.get("context"):
        return {
            "answer": "I can’t answer that yet without spoilers (or I don’t have enough context from earlier chapters).",
            "debug": resp
        }

    prompt = build_prompt(
        resp["question"],
        resp["current_chapter"],
        resp["context"]
    )

    model_answer = ollama_chat(prompt["system"], prompt["user"], model=model)

    retrieved_chapters = sorted({
        int(c) for c in re.findall(r"\[Chapter (\d+)\]", resp["context"])
    })

    return {
        "answer": model_answer,
        "debug": {
            "retrieved_chapters": retrieved_chapters,
            "context_chars": len(resp["context"]),
            "model": model
        }
    }

In [9]:
from IPython.display import display, Markdown

STATE = {
    "book": "NOTW",
    "chapter": 9,
    "model": "mistral",
    "debug": False,
    "show_sources": True,
}

def _md(s: str):
    display(Markdown(s))

def _banner():
    _md(
        f"""
## 🍺 The Waystone Companion
<small>
Book: {STATE['book']} &nbsp;•&nbsp;
Chapter: {STATE['chapter']} &nbsp;•&nbsp;
Model: {STATE['model']}
</small>

> *It was night again. The Waystone Inn lay in silence.*
---
"""
    )

def _help():
    _md(
        """
**Commands**
- `/chapter N` — set spoiler boundary
- `/model NAME` — set local model
- `/status` — show current settings
- `/debug on|off` — toggle debug
- `/sources on|off` — toggle sources
- `/help` — show commands
- `/quit` — leave the Waystone
"""
    )

def _status():
    _md(
        f"""
**Status**
- 📖 Book: `{STATE['book']}`
- 📘 Chapter: `{STATE['chapter']}`
- 🤖 Model: `{STATE['model']}`
- 🧪 Debug: `{STATE['debug']}`
- 🧩 Sources: `{STATE['show_sources']}`
"""
    )

def _parse_cmd(q: str):
    parts = q.strip().split()
    return parts[0].lower(), parts[1:]

def _kvothe_wrap(text: str) -> str:
    text = text.strip()
    if not text:
        return "*Kvothe is quiet for a moment, then shakes his head.*"

    return (
        "_Kvothe rests his forearms on the bar, voice low and even:_\n\n"
        f"{text}"
    )

def chat():
    _banner()
    _help()

    while True:
        q = input("🟩 You: ").strip()
        if not q:
            continue

        # Commands
        if q.startswith("/"):
            cmd, args = _parse_cmd(q)

            if cmd in {"/quit", "/exit"}:
                _md("*The fire crackles softly as the room settles back into silence.*")
                break

            if cmd == "/help":
                _help()
                continue

            if cmd == "/status":
                _status()
                continue

            if cmd == "/chapter":
                try:
                    STATE["chapter"] = int(args[0])
                    _md(f"✅ Chapter set to **{STATE['chapter']}**")
                except Exception:
                    _md("⚠️ Usage: `/chapter 37`")
                continue

            if cmd == "/model":
                if not args:
                    _md("⚠️ Usage: `/model mistral`")
                else:
                    STATE["model"] = " ".join(args)
                    _md(f"✅ Model set to **{STATE['model']}**")
                continue

            if cmd == "/debug":
                if not args or args[0].lower() not in {"on", "off"}:
                    _md("⚠️ Usage: `/debug on` or `/debug off`")
                else:
                    STATE["debug"] = (args[0].lower() == "on")
                    _md(f"✅ Debug set to **{STATE['debug']}**")
                continue

            if cmd == "/sources":
                if not args or args[0].lower() not in {"on", "off"}:
                    _md("⚠️ Usage: `/sources on` or `/sources off`")
                else:
                    STATE["show_sources"] = (args[0].lower() == "on")
                    _md(f"✅ Sources set to **{STATE['show_sources']}**")
                continue

            _md("⚠️ Unknown command. Type `/help`.")
            continue

        # Normal question
        out = answer(q, current_chapter=STATE["chapter"], model=STATE["model"])
        ans = out.get("answer", "").strip()
        dbg = out.get("debug", {}) or {}

        _md(f"### 🟦 Kvothe\n{_kvothe_wrap(ans)}")

        if STATE["show_sources"] and dbg.get("retrieved_chapters") is not None:
            _md(f"<small>📌 Sources: `{dbg.get('retrieved_chapters')}`</small>")

        if STATE["debug"] and dbg:
            _md(
                f"<small>🧪 debug — context_chars: `{dbg.get('context_chars')}` | model: `{dbg.get('model')}`</small>"
            )

        _md("---")

In [10]:
chat()


## 🍺 The Waystone Companion
<small>
Book: NOTW &nbsp;•&nbsp;
Chapter: 9 &nbsp;•&nbsp;
Model: mistral
</small>

> *It was night again. The Waystone Inn lay in silence.*
---



**Commands**
- `/chapter N` — set spoiler boundary
- `/model NAME` — set local model
- `/status` — show current settings
- `/debug on|off` — toggle debug
- `/sources on|off` — toggle sources
- `/help` — show commands
- `/quit` — leave the Waystone


🟩 You:  where is the waystone inn?


### 🟦 Kvothe
_Kvothe rests his forearms on the bar, voice low and even:_

From the context provided, it is not explicitly stated where the Waystone Inn is located. However, in Chapter 1 (expanded), it is mentioned that if there had been a wind, it would have come from the trees outside the inn and brushed down the road, implying that the inn is situated along some sort of road or path. In Chapter 3 (expanded), Bast brings something in from outside to the bar, suggesting that the Waystone Inn has an outdoor area as well. However, without further context or information, I cannot provide a more specific location for the Waystone Inn.

<small>📌 Sources: `[0, 1, 3]`</small>

...

### 🟦 Kvothe
_Kvothe rests his forearms on the bar, voice low and even:_

In the provided context from The Kingkiller Chronicle up to Chapter 9, the Chandrian are not directly mentioned or described. However, there are hints that suggest their presence:

1. Taborlin, who is known for his magical abilities, fell, and it is implied he may have fallen because of the Chandrian (Chapter 1). The Chandrian are feared entities that can hunt down and kill magic users.
2. In Chapter 7, Chronicler mentions interviewing Oren Velciter, who sought him out. Oren Velciter is a legendary figure known for his tales about the Chandrian, implying their importance in the story's world.
3. The fear of the Chandrian might be related to the "dangerous stuff" Kote mentions in Chapter 9, which could be an antidote against their poisonous touch (implied but not confirmed).

<small>📌 Sources: `[1, 3, 7, 9]`</small>

---

🟩 You:  /exit


*The fire crackles softly as the room settles back into silence.*